In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

In [ ]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

membagi train set dan validation set

In [ ]:
import os
import shutil

# Path ke direktori utama dataset
base_dir = '/tmp/rockpaperscissors'

# Membuat folder baru untuk train set dan validation set
train_dir = os.path.join(base_dir, 'train')
os.makedirs(train_dir, exist_ok=True)
validation_dir = os.path.join(base_dir, 'validation')
os.makedirs(validation_dir, exist_ok=True)

# Melakukan iterasi pada setiap kelas ('rock', 'paper', 'scissors')
classes = ['rock', 'paper', 'scissors']

for class_name in classes:
    class_dir = os.path.join(base_dir, class_name)
    train_class_dir = os.path.join(train_dir, class_name)
    validation_class_dir = os.path.join(validation_dir, class_name)

    # Membuat folder train dan validation untuk setiap kelas
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(validation_class_dir, exist_ok=True)

    # Mendapatkan daftar file dalam folder kelas
    files = os.listdir(class_dir)
    n_val = int(0.4 * len(files))  # 40% untuk validation

    # Memindahkan file ke folder validation
    for file in files[:n_val]:
        src_path = os.path.join(class_dir, file)
        dest_path = os.path.join(validation_class_dir, file)
        shutil.move(src_path, dest_path)

    # Memindahkan file ke folder train
    for file in files[n_val:]:
        src_path = os.path.join(class_dir, file)
        dest_path = os.path.join(train_class_dir, file)
        shutil.move(src_path, dest_path)

# Menampilkan struktur direktori
print("Train Directory Structure:")
print(os.listdir(train_dir))

print("\nValidation Directory Structure:")
print(os.listdir(validation_dir))


Augmentasi Gambar menggunakan ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping]
)

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

# Mengunggah gambar menggunakan modul files
uploaded = files.upload()

# Melakukan prediksi untuk setiap gambar yang diunggah
for fn in uploaded.keys():
    # Memprediksi gambar
    path = fn
    img = image.load_img(path, target_size=(150, 150))

    plt.imshow(img)
    plt.axis('off')
    plt.show()

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = model.predict(images, batch_size=10)

    print(f'File: {fn}')

    # Menentukan kelas berdasarkan nilai prediksi tertinggi
    predicted_class = np.argmax(classes)

    if predicted_class == 0:
        print('Predicted Class: Paper')
    elif predicted_class == 1:
        print('Predicted Class: Rock')
    else:
        print('Predicted Class: Scissors')

In [ ]:
print(train_generator.class_indices)